# Importing Spark

In [1]:
# install Java JDK 8
# install SPARK
# install pyspark
# eventually consider pyspark
# eventuelly consider SPARK_HOME, or SPARK_PATH, etc.
!pip install pyspark

     |████████████████████████████████| 212.4 MB 55 kB/s 
     |████████████████████████████████| 198 kB 56.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=da989ff650987d5c020ad84663c3635f0eb94032753d267e26728b4d28db9a69
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("Python Spark").getOrCreate()

# Loading the data

In [2]:
df_ratings = spark.read\
    .option("delimiter", "\t")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('u.data')

In [3]:
# print the dataframe schema
df_ratings.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [4]:
# show a sample of the data (the dataframe executes the whole pipeline at this stage)
df_ratings.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [5]:
# set the rdd equivalent of the dataframe
rdd_ratings = df_ratings.rdd

# Basic Queries

#### Exercice 1 - Number of movies per user (using RDD then Dataframe)

Calculer pour chaque utilisateur le nombre de films notés, et afficher le résultat pour l'un d'entre eux. Utilisez dans un premier temps les RDD puis les Dataframes.

In [6]:
rdd_ratings

MapPartitionsRDD[18] at javaToPython at NativeMethodAccessorImpl.java:0

In [10]:
# classical RDD approach
result_1 = rdd_ratings.map(lambda r: (r[0], 1)).reduceByKey(lambda v1, v2: v1+v2).take(1)
print(result_1)

[(196, 39)]


In [12]:
# dataframe approach, filtering on the previous user to compare results
df_ratings.filter(df_ratings.user_id == result_1[0][0]).groupBy('user_id').count().show(1)

+-------+-----+
|user_id|count|
+-------+-----+
|    196|   39|
+-------+-----+



#### Exercice 2 - Average rating per user (using RDD then Dataframe)

Calculer pour chaque utilisateurs la note moyenne donnée et afficher le résultat pour l'un d'entre eux. Utilisez dans un premier temps les RDD puis les Dataframes.

In [30]:
# RDD
# classical RDD approach
rdd_map = rdd_ratings.map(lambda r: (r[0], int(r[2])))
rdd_agg = rdd_map.aggregateByKey(
    (0, 0), # initialise les valeurs pour chacun des users
    lambda a,b: (a[0] + b,    a[1] + 1), # combine deux lignes
    lambda a,b: (a[0] + b[0], a[1] + b[1]) # combine deux partitions
)

rdd_result = rdd_agg.mapValues(lambda v: float(v[0])/v[1])

result_1 = rdd_result.take(1)
print(result_1)

[(196, 3.6153846153846154)]


In [32]:
# dataframe approach
df_ratings.filter(df_ratings.user_id == result_1[0][0]).groupBy("user_id").mean().show(1) 
#or

from pyspark.sql.functions import avg

df_ratings.filter(df_ratings['user_id']==result_1[0][0])\
    .groupBy('user_id')\
    .agg(avg('rating'))\
    .show(1)

+-------+------------+-----------------+------------------+-------------------+
|user_id|avg(user_id)|     avg(item_id)|       avg(rating)|     avg(timestamp)|
+-------+------------+-----------------+------------------+-------------------+
|    196|       196.0|375.8205128205128|3.6153846153846154|8.812517328461539E8|
+-------+------------+-----------------+------------------+-------------------+

+-------+------------------+
|user_id|       avg(rating)|
+-------+------------------+
|    196|3.6153846153846154|
+-------+------------------+



#### Exercice 3 - Top-5 movies with at least 15 votes (Dataframe)

Afficher les 5 meilleurs films parmi ceux qui ont reçu au moins 15 votes.

*Indices:*
* Utiliser df_ratings pour calculer la moyenne, filtrer les films qui ont moins de 15 notes et classer les films par ordre décroissant.
* Faire un join avec df_movies pour afficher le nom des films sélectionnés.

In [33]:
df_items = spark.read\
    .option("delimiter", "|")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('u.item')

In [34]:
df_items.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- video_release_date: string (nullable = true)
 |-- IMDb_url: string (nullable = true)
 |-- unknown: integer (nullable = true)
 |-- action: integer (nullable = true)
 |-- adventure: integer (nullable = true)
 |-- animation: integer (nullable = true)
 |-- children\s: integer (nullable = true)
 |-- comedy: integer (nullable = true)
 |-- crime: integer (nullable = true)
 |-- documentary: integer (nullable = true)
 |-- drama: integer (nullable = true)
 |-- fantasy: integer (nullable = true)
 |-- film-noir: integer (nullable = true)
 |-- horror: integer (nullable = true)
 |-- musical: integer (nullable = true)
 |-- mystery: integer (nullable = true)
 |-- romance: integer (nullable = true)
 |-- sci-fi: integer (nullable = true)
 |-- thriller: integer (nullable = true)
 |-- war: integer (nullable = true)
 |-- western: integer (nullable = true)



In [35]:
df_items.show(1)

+--------+----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|     movie_title|release_date|video_release_date|            IMDb_url|unknown|action|adventure|animation|children\s|comedy|crime|documentary|drama|fantasy|film-noir|horror|musical|mystery|romance|sci-fi|thriller|war|western|
+--------+----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1|Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
+--------+----------------+------------+------------------+-

In [49]:
df_ratings.filter(df_ratings.groupBy("item_id").count()["count"] > 15)


AnalysisException: ignored